<a href="https://colab.research.google.com/github/marciogsantana/Ciencia-de-dados/blob/main/streaming_read_from_file_over_Event_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Streaming de Dados em Arquivos Sobre Event-Time**

In [ ]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import window
from pyspark.sql.functions import desc, asc




spark = SparkSession \
    .builder \
    .appName("StructuredCountFile") \
    .getOrCreate()

In [ ]:
#definindo um esquema para os dados
from pyspark.sql.types import StructType


userSchema = StructType().add("timestamp", "timestamp").add("word", "string")

In [ ]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas dos arquivos adicionados no diretório
files_dir = spark.readStream\
    .format("csv")\
    .schema(userSchema)\
    .option('includeTimestamp', 'true')\
    .option("header", "true")\
    .option("delimiter", ";")\
    .option("maxFilesPerTrigger", 1)\
    .load("/home/tulio/Documents/Aulas_PFC/arquivos_csv/*.csv")

In [ ]:
#print do esquema
files_dir.printSchema

<bound method DataFrame.printSchema of DataFrame[timestamp: timestamp, word: string]>

In [ ]:
# Divide as linhas recebidas em cada palavra
words = files_dir.select(
   explode(
       split(files_dir.word, " ")
   ).alias("word"), files_dir.timestamp
)


In [ ]:
words.isStreaming

True

In [ ]:
#agrupa os dados através da janela de tempo e computa sobre cada um dos grupos
windowedCounts = words.groupBy(
    window(words.timestamp, "10 minutes", "5 minutes"),
    words.word
).count()#.sort(asc("window"))

In [ ]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
query = windowedCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option('truncate', 'false')\
    .start()


query.awaitTermination() #aguarda até que a "streaming query" termine 